In [89]:
# relevant packages & modules
import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time
import requests
import urllib.request
import json
from tqdm import tqdm
# from tqdm.notebook import tqdm_notebook as tqdm
import undetected_chromedriver.v2 as uc


# Have powerfull bot protection, so we need undetected_chrome
class ScrapperPlusvalia:
    
    driver = None

    
    logged_in = False
    
    def __init__(self):
        
        self.driver = uc.Chrome(version_main=95)
        self.root = "https://www.plusvalia.com/inmuebles-ordenado-por-fechaonline-descendente"
        self.base = f"{self.root}-pagina-"
        self.driver.get(f"{self.base}1.html")

    def wait(self, t):
        time.sleep(t)
        
    def check_requisites(self):     
        return True
    
    def collect_urls(self,page_from,page_to,waiting_time=2)->list:
        
        if not self.check_requisites():
            return []

        urls = []
        repeated = []


        url = f"{self.base}{page_from}.html"
        self.driver.get(url)

        with open("urls.txt","a") as f:

            for page_i in tqdm(range(page_from, page_to+1)):
                
                chunk = self.driver.find_elements(By.XPATH, '//div[contains(@class,"list-card-container")]/div')
                
                for c in chunk:
                    if str(c.get_attribute("class")) != "ads-container":

                        url_item = str(c.get_attribute("data-to-posting"))
                        if url_item:
                            if url_item not in urls:
                                urls.append(url_item)
                            else:
                                repeated.append(f"Repeated inmuebles-ordenado-por-fechaonline-descendente-{page_i}.html:{url_item}")

                f.write("\n".join(map(lambda s: f"https://www.plusvalia.com{s}", urls)))
                urls = []

                current_url = self.driver.current_url


                if current_url == f"{self.root}.html":
                    page = 1
                else:
                    page = int(re.match(r"^https\:\/\/www\.plusvalia\.com\/[A-z\-]*(\d+)\.html$", current_url).groups()[0])

                if page == page_to:
                    break
                
                next = self.driver.find_elements(By.CSS_SELECTOR, 'li.pag-go-next > a')[0]
                next.click()
                self.wait(waiting_time)

        return urls, repeated
    
    def close(self):
        if self.driver:
            self.driver.close()
            
    def __del__(self):
        print("Closing...")
        self.close()

In [90]:
scrapper = ScrapperPlusvalia()

Exception ignored in: <function ScrapperPlusvalia.__del__ at 0x7f86cccec280>
Traceback (most recent call last):
  File "/tmp/ipykernel_29198/515809178.py", line 90, in __del__
  File "/tmp/ipykernel_29198/515809178.py", line 86, in close
  File "/home/magody/anaconda3/lib/python3.8/site-packages/selenium/webdriver/remote/webdriver.py", line 937, in close
    self.execute(Command.CLOSE)
  File "/home/magody/anaconda3/lib/python3.8/site-packages/selenium/webdriver/remote/webdriver.py", line 424, in execute
    self.error_handler.check_response(response)
  File "/home/magody/anaconda3/lib/python3.8/site-packages/selenium/webdriver/remote/errorhandler.py", line 247, in check_response
    raise exception_class(message, screen, stacktrace)
selenium.common.exceptions.WebDriverException: Message: chrome not reachable
  (Session info: chrome=96.0.4664.110)
Stacktrace:
#0 0x5568fb81c463 (/home/magody/.local/share/undetected_chromedriver/chromedriver (deleted)+0xffffffffff720462)
#1 0x5568fb2f450

Closing...


In [92]:
urls, _ = scrapper.collect_urls(1,3468,1.5)  # 3468

#with open("urls_repeated.txt","w") as f:
#    f.write("\n".join(repeated))

  2%|▏         | 54/3468 [02:10<2:17:34,  2.42s/it]


KeyboardInterrupt: 